In [1]:
import pandas as pd
import torch
import math
import torch.nn.functional as F
from torch import nn, optim

from torch.utils.data import TensorDataset, DataLoader

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import numpy as np
from pynvml import *
import time


d:\OneDrive\Code\Python\Training\ML\Tests\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [91]:
#data Wokring functions:
def imputPd(df, columnName, imputeObj):
    df[columnName]= imputeObj.fit_transform(df[[columnName]]).ravel()
    return df

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("CUDA Available: ", dev)


#load Data********
testData = pd.read_csv('titanic/test.csv')
trainData = pd.read_csv('titanic/train.csv')
trainData = trainData.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
testData = testData.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
# trainData.head()



#Impute any Null Values if there are any null values
trainDataNulls = trainData.columns[trainData.isna().any()].tolist()
testDataNulls = testData.columns[testData.isna().any()].tolist()
# print(testDataNulls,testDataNulls)
imp = SimpleImputer(strategy="most_frequent")

if not len(trainDataNulls) == 0:
    for colName in trainDataNulls:
        trainData = imputPd(trainData, colName, imp)

if not len(testDataNulls) == 0:
    for colName in testDataNulls:
        testData = imputPd(testData, colName, imp)


#split out dependent and indepent vairables:
x_train = trainData.drop('Survived', axis=1).copy()
y_train =  trainData['Survived'].copy()
x_test = testData.copy()
# x_train.head()



#convert text Labels to Numaric:
leSexTrain = preprocessing.LabelEncoder()
leSexTest = preprocessing.LabelEncoder()
x_train['Sex'] = leSexTrain.fit_transform(x_train['Sex'])
x_test['Sex'] = leSexTest.fit_transform(x_test['Sex'])

leEmbarkedTrain = preprocessing.LabelEncoder()
leEmbarkedTest = preprocessing.LabelEncoder()
x_train['Embarked'] = leEmbarkedTrain.fit_transform(x_train['Embarked'])
x_test['Embarked'] = leEmbarkedTest.fit_transform(x_test['Embarked'])




#convert Data from Dataframe to tensor
x_trainTen = torch.tensor(x_train.values, device=dev, dtype=torch.float64)
# x_trainTen = x_trainTen.type(torch.LongTensor)
# x_trainTen = x_trainTen.to(device)


y_trainTen = torch.tensor(y_train.values, device=dev, dtype=torch.long)
# y_trainTen = y_trainTen.type(torch.LongTensor)
# y_trainTen = y_trainTen.to(device)

x_testTen = torch.tensor(x_test.values, device=dev, dtype=torch.float64)
# x_testTen = x_testTen.type(torch.LongTensor)
# x_testTen = x_testTen.to(device)

#load data into PyTorch dataset
fullTrain_ds = TensorDataset(x_trainTen, y_trainTen)

#Split trainging data into train and validate sets
train_ds, valid_ds = torch.utils.data.random_split(fullTrain_ds, [.8, .2], generator=torch.Generator().manual_seed(42))
print(f'fullDS: {len(fullTrain_ds)},  TrainDS:{len(train_ds)},   ValidDS:{len(valid_ds)}')


CUDA Available:  cuda
fullDS: 891,  TrainDS:713,   ValidDS:178


In [92]:
#general Functions and classes for nn:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        # self.linear1 = nn.Linear(12, 12)
        self.linear1 = nn.Linear(7, 7, device=dev, dtype=torch.float64)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(7, 5, device=dev, dtype=torch.float64)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(5, 2, device=dev, dtype=torch.float64)
        # self.softmax = nn.Softmax(dim=1)
        # self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, xb):
        lin1_out = self.linear1(xb)
        sigmoid_out1 = self.sigmoid1(lin1_out)
        sigmoid_out2 = self.sigmoid2(self.linear2(sigmoid_out1))
        # lastLayer = self.softmax(self.linear3(sigmoid_out2))
        # lastLayer = torch.flatten(lastLayer, 0)
        return self.linear3(sigmoid_out2)



def loss_batch(model, loss_func, xb, yb, opt=None):
    pred = model(xb)
    pred = torch.squeeze(pred, 0)
    loss = loss_func(pred, yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)
    


def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        if epoch%100 == 99:
            print(epoch, val_loss, print_gpu_utilization())


def get_data(train_ds, bs):
    return DataLoader(train_ds, batch_size=bs, shuffle=True),
    

In [105]:

bs = 64
# bs = 64
epochs = 10000
lr = .01

train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

loss_func = F.cross_entropy
# loss_func = F.mse_loss

# model, opt = get_model()
model = Mnist_Logistic()
model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr)
# opt = optim.Adam(model.parameters(), lr=lr)

start_time = time.time()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)
print("--- %s seconds ---" % (time.time() - start_time))


GPU memory occupied: 2019 MB.
99 0.6265355549789884 None
GPU memory occupied: 2012 MB.
199 0.621718753027748 None
GPU memory occupied: 2010 MB.
299 0.6149312803255954 None
GPU memory occupied: 2015 MB.
399 0.616373578957956 None
GPU memory occupied: 1973 MB.
499 0.6264705212321665 None
GPU memory occupied: 1968 MB.
599 0.618307961059803 None
GPU memory occupied: 1961 MB.
699 0.6190673011365003 None
GPU memory occupied: 1956 MB.
799 0.618429914713917 None
GPU memory occupied: 1950 MB.
899 0.6181004967250859 None
GPU memory occupied: 1950 MB.
999 0.6154121546209899 None
GPU memory occupied: 1955 MB.
1099 0.611314392874266 None
GPU memory occupied: 1958 MB.
1199 0.6115306343744991 None
GPU memory occupied: 1958 MB.
1299 0.6077044019461962 None
GPU memory occupied: 1958 MB.
1399 0.5933972155231606 None
GPU memory occupied: 1958 MB.
1499 0.5890516604853812 None
GPU memory occupied: 1954 MB.
1599 0.583072642206988 None
GPU memory occupied: 1946 MB.
1699 0.5459298976208216 None
GPU memory occ

In [67]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()